In [1]:
ROOT = "/content/drive/MyDrive/Colab Notebooks"

In [2]:
import os
os.chdir(ROOT)
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
plt.style.use('fivethirtyeight')
%load_ext tensorboard

In [5]:
(x_train, y_train) , (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train/255.0
x_test = x_test /255.0
x_valid = x_train[:5000]
x_train_full = x_train[5000:]
y_valid = y_train[:5000]
y_train_full = y_train[5000:]

11501568/11490434 [==============================] - 0s 0us/step


traning model which we will use for transfer learning for below

In [7]:
tf.random.set_seed(42)
np.random.seed(42)

layers = [tf.keras.layers.Flatten( input_shape = [28, 28]),
          tf.keras.layers.Dense(300, kernel_initializer= 'he_normal'),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(100, kernel_initializer= 'he_normal'),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(10, activation= 'softmax'),
          ]
model = tf.keras.models.Sequential(layers)

In [8]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.SGD(lr = 1e-3),
              metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [10]:
histry = model.fit(x_train, y_train, epochs = 10, validation_data = (x_valid, y_valid), verbose=2)

Epoch 1/10
1875/1875 - 7s - loss: 1.4750 - accuracy: 0.6152 - val_loss: 0.8821 - val_accuracy: 0.8128 - 7s/epoch - 4ms/step
Epoch 2/10
1875/1875 - 4s - loss: 0.7026 - accuracy: 0.8362 - val_loss: 0.5542 - val_accuracy: 0.8700 - 4s/epoch - 2ms/step
Epoch 3/10
1875/1875 - 4s - loss: 0.5153 - accuracy: 0.8673 - val_loss: 0.4453 - val_accuracy: 0.8876 - 4s/epoch - 2ms/step
Epoch 4/10
1875/1875 - 4s - loss: 0.4403 - accuracy: 0.8820 - val_loss: 0.3936 - val_accuracy: 0.8968 - 4s/epoch - 2ms/step
Epoch 5/10
1875/1875 - 4s - loss: 0.3990 - accuracy: 0.8909 - val_loss: 0.3596 - val_accuracy: 0.9044 - 4s/epoch - 2ms/step
Epoch 6/10
1875/1875 - 5s - loss: 0.3721 - accuracy: 0.8963 - val_loss: 0.3376 - val_accuracy: 0.9100 - 5s/epoch - 2ms/step
Epoch 7/10
1875/1875 - 4s - loss: 0.3527 - accuracy: 0.9010 - val_loss: 0.3211 - val_accuracy: 0.9148 - 4s/epoch - 2ms/step
Epoch 8/10
1875/1875 - 5s - loss: 0.3378 - accuracy: 0.9050 - val_loss: 0.3080 - val_accuracy: 0.9162 - 5s/epoch - 3ms/step
Epoch 9/

In [11]:
model.save('full_model_mnist.h5')

Transfer_Learning

In [12]:
load_model = tf.keras.models.load_model('full_model_mnist.h5')

In [13]:
load_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [15]:
for layer in load_model.layers:
  print('layer :', layer.name)

layer : flatten_1
layer : dense_3
layer : leaky_re_lu_2
layer : dense_4
layer : leaky_re_lu_3
layer : dense_5


In [19]:
for layer in load_model.layers[:-1]:
  layer.trainable = False
  print('layer :', layer.name ,    layer.trainable)

layer : flatten_1 False
layer : dense_3 False
layer : leaky_re_lu_2 False
layer : dense_4 False
layer : leaky_re_lu_3 False


In [23]:
lower_pretained_layers = load_model.layers[:-1]
new_model = tf.keras.models.Sequential(lower_pretained_layers)
new_model.add(tf.keras.layers.Dense(2, activation = 'softmax'))

In [24]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_6 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

here we have taken previous models pre trained layers here as a transfer laerning to reduce computation

In [25]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

y dataset has converted into binary

In [27]:
def update_enev_odd(labels):
  for i , j in enumerate(labels):
    labels[i] = np.where(j % 2 == 0, 1, 0)
  return labels

In [29]:
y_train_bin, y_test_bin, y_valid_bin = update_enev_odd([y_train, y_test, y_valid])

In [31]:
new_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.SGD(lr = 1e-3),
              metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [32]:
np.unique(y_train_bin)

array([0, 1])

In [33]:
new_history = new_model.fit(x_train, y_train_bin, epochs = 10, 
                            validation_data = (x_valid, y_valid_bin), verbose =2)

Epoch 1/10
1875/1875 - 5s - loss: 0.3820 - accuracy: 0.8348 - val_loss: 0.3185 - val_accuracy: 0.8698 - 5s/epoch - 3ms/step
Epoch 2/10
1875/1875 - 4s - loss: 0.3244 - accuracy: 0.8624 - val_loss: 0.3004 - val_accuracy: 0.8772 - 4s/epoch - 2ms/step
Epoch 3/10
1875/1875 - 4s - loss: 0.3108 - accuracy: 0.8692 - val_loss: 0.2896 - val_accuracy: 0.8824 - 4s/epoch - 2ms/step
Epoch 4/10
1875/1875 - 4s - loss: 0.3026 - accuracy: 0.8732 - val_loss: 0.2843 - val_accuracy: 0.8850 - 4s/epoch - 2ms/step
Epoch 5/10
1875/1875 - 4s - loss: 0.2967 - accuracy: 0.8763 - val_loss: 0.2780 - val_accuracy: 0.8882 - 4s/epoch - 2ms/step
Epoch 6/10
1875/1875 - 4s - loss: 0.2920 - accuracy: 0.8782 - val_loss: 0.2732 - val_accuracy: 0.8884 - 4s/epoch - 2ms/step
Epoch 7/10
1875/1875 - 4s - loss: 0.2881 - accuracy: 0.8801 - val_loss: 0.2698 - val_accuracy: 0.8904 - 4s/epoch - 2ms/step
Epoch 8/10
1875/1875 - 4s - loss: 0.2847 - accuracy: 0.8817 - val_loss: 0.2674 - val_accuracy: 0.8926 - 4s/epoch - 2ms/step
Epoch 9/

In [50]:
new_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.SGD(lr = 1e-3),
              metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


prediction

In [51]:
x_new = x_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [38]:
np.argmax(new_model.predict(x_new), axis=1)

array([0, 1, 0])

applying transfer learning by converting y dataset in binary by condition if i > 5 then return 1 otherwise i =0

In [44]:
def new_data(lab):
  for i, j in enumerate(lab):
    lab[i] = np.where(j > 5, 1, 0)
  return lab
y_train_five, y_test_five, y_valid_five = new_data([y_train, y_test, y_valid])

In [69]:
np.unique(y_test_five)

array([0, 1])

useing transfer learining

In [58]:
lower_pretained_layers = load_model.layers[:-1]
new_model_five = tf.keras.models.Sequential(lower_pretained_layers)
new_model_five.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

compiling model

In [61]:
new_model_five.compile(loss = 'binary_crossentropy',
              optimizer = tf.keras.optimizers.SGD(lr = 1e-3),
              metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


traning the model

In [62]:
new_history = new_model_five.fit(x_train, y_train_five, epochs = 10, 
                            validation_data = (x_valid, y_valid_five), verbose =2)

Epoch 1/10
1875/1875 - 5s - loss: 0.5599 - accuracy: 0.7303 - val_loss: 0.4901 - val_accuracy: 0.7852 - 5s/epoch - 3ms/step
Epoch 2/10
1875/1875 - 4s - loss: 0.4651 - accuracy: 0.7969 - val_loss: 0.4367 - val_accuracy: 0.8116 - 4s/epoch - 2ms/step
Epoch 3/10
1875/1875 - 4s - loss: 0.4277 - accuracy: 0.8212 - val_loss: 0.4057 - val_accuracy: 0.8300 - 4s/epoch - 2ms/step
Epoch 4/10
1875/1875 - 4s - loss: 0.4047 - accuracy: 0.8340 - val_loss: 0.3848 - val_accuracy: 0.8438 - 4s/epoch - 2ms/step
Epoch 5/10
1875/1875 - 4s - loss: 0.3890 - accuracy: 0.8429 - val_loss: 0.3702 - val_accuracy: 0.8482 - 4s/epoch - 2ms/step
Epoch 6/10
1875/1875 - 4s - loss: 0.3773 - accuracy: 0.8481 - val_loss: 0.3594 - val_accuracy: 0.8544 - 4s/epoch - 2ms/step
Epoch 7/10
1875/1875 - 4s - loss: 0.3683 - accuracy: 0.8529 - val_loss: 0.3506 - val_accuracy: 0.8584 - 4s/epoch - 2ms/step
Epoch 8/10
1875/1875 - 4s - loss: 0.3611 - accuracy: 0.8561 - val_loss: 0.3432 - val_accuracy: 0.8618 - 4s/epoch - 2ms/step
Epoch 9/

In [67]:
x_five = x_test[:3]

y_test[:10], y_test_five[:10]

(array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8),
 array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1]))

In [68]:
np.round(new_model.predict(x_five))

array([[0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)